In [1]:
import os
import pandas as pd
import numpy as np
pd.set_option("display.max_colwidth", 2)

os.chdir("/home/coinse/Workspace/SBFL")

# Load bug info

In [2]:
from scripts.util import load_bug_data, find_covmat_func, find_covmat_idx
from functools import reduce
from typing import Set

pid, vid = "wireshark", 6

bug = load_bug_data(pid, vid)

print(f"{bug.buggy_files=}")
print(f"{bug.buggy_lines=}")
print("bug.coverage_df=")
display(bug.coverage_df)
print(f"{bug.failing_tests=}")
print(f"{bug.failing_tests_GT=}")
print("bug.functions={", end="")
for func in list(bug.functions)[:5]:
    print(f"{func},", end=" ")
print(", ...}")
print(f"{bug.id=}")
print(f"{bug.num_test_cases=}")
print(f"{bug.passing_tests=}")
print(f"{bug.path_to_cov_df=}")
print(f"{bug.path_to_patch=}")
print(f"{bug.path_to_test_results=}")
print("bug.source_files=", end="")
for src in list(bug.source_files)[:5]:
    print(f"{src},", end=" ")
print(", ...}")

fault_idxs = sorted(bug.buggy_lines)
# fault_idx2meth = {}
fault_meth2idx = {}
err_msg = ""
for fault_idx in fault_idxs:
    succ_func, ret_func = find_covmat_func(
        pid, bug.coverage_df, fault_idx, debug=False
    )
    if succ_func:
        succ_idx, ret_idx = find_covmat_idx(
            pid, bug.coverage_df, fault_idx, debug=False
        )
        if succ_idx:
            if ret_func in fault_meth2idx:
                fault_meth2idx[ret_func].add(ret_idx)
            else:
                fault_meth2idx[ret_func] = {ret_idx}
        else:
            print(f"{ret_idx}")
            if ret_func not in fault_meth2idx:
                fault_meth2idx[ret_func] = set()
    else:
        print(f"{ret_func}")
# fault_meths: Set = reduce(
#     lambda x, y: x | set([y]), fault_idx2meth.values(), set()
# )
# fault_meth2idx = {
#     v: {idx for idx in fault_idx2meth if fault_idx2meth[idx] == v}
#     for v in fault_meths
# }
fault_meths = set(fault_meth2idx.keys())
for meth in fault_meth2idx:
    print(f"{meth=}: {fault_meth2idx[meth]}")
# print(f"{fault_meths=}")


bug.buggy_files={'epan/wslua/wslua_byte_array.c'}
bug.buggy_lines={('epan/wslua/wslua_byte_array.c', 267), ('epan/wslua/wslua_byte_array.c', 266), ('epan/wslua/wslua_byte_array.c', 262), ('epan/wslua/wslua_byte_array.c', 268), ('epan/wslua/wslua_byte_array.c', 265), ('epan/wslua/wslua_byte_array.c', 261)}
bug.coverage_df=


33  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   556   
                                                                                  177   556   
                                                                                  199   556   
                                                                                  203   556   
                                                                                  205   556   
...                                                                                     ...   
/home/workspace/epan/dissectors/packet-isns.c         proto_reg_handoff_isns      1712  556   
                                                                                  1713  556   
                                                                                  1715  556   
                                                                                  1716  556   
                                                                                  1717  556   

                                                                                         32  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   554   
                                                                                  177   554   
                                                                                  199   554   
                                                                                  203   554   
                                                                                  205   554   
...                                                                                     ...   
/home/workspace/epan/dissectors/packet-isns.c         proto_reg_handoff_isns      1712  554   
                                                                                  1713  554   
                                                                                  1715  554   
                                                                                  1716  554   
                                                                                  1717  554   

                                                                                          9  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   278   
                                                                                  177   278   
                                                                                  199   278   
                                                                                  203   278   
                                                                                  205   278   
...                                                                                     ...   
/home/workspace/epan/dissectors/packet-isns.c         proto_reg_handoff_isns      1712  278   
                                                                                  1713  278   
                                                                                  1715  278   
                                                                                  1716  278   
                                                                                  1717  278   

                                                                                         10  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   280   
                                                                                  177   280   
 

bug.failing_tests={'34'}
bug.failing_tests_GT={'34'}
bug.functions={('/home/workspace/epan/dissectors/packet-tls-utils.c', 'ssl3_check_mac'), ('/home/workspace/epan/dissectors/packet-usbip.c', 'proto_reg_handoff_usbip'), ('/home/workspace/epan/wmem/wmem_strutl.c', 'wmem_strdup_printf'), ('/home/workspace/epan/proto.c', 'proto_tree_add_double_format'), ('/home/workspace/wiretap/file_wrappers.c', 'file_clearerr'), , ...}
bug.id='wireshark-6'
bug.num_test_cases=34
bug.passing_tests={'3', '15', '29', '16', '11', '2', '27', '13', '22', '26', '17', '25', '32', '23', '24', '12', '4', '8', '1', '19', '18', '30', '14', '33', '28', '7', '9', '31', '20', '6', '5', '10', '21'}
bug.path_to_cov_df='/home/coinse/Workspace/defects4cpp/coverage/wireshark_6_buggy/coverage_df.pkl'
bug.path_to_patch='/home/coinse/Workspace/defects4cpp/defects4cpp/taxonomy/wireshark/patch/0006-buggy.patch'
bug.path_to_test_results='/home/coinse/Workspace/defects4cpp/coverage/wireshark_6_buggy/test_results.json'
bug.source_

# Get spectra

In [3]:
covmat = bug.coverage_df
pass_tcs = bug.passing_tests
fail_tcs = bug.failing_tests

covmat_bool = covmat.astype(bool)
P = len(pass_tcs)
F = len(fail_tcs)
e_p = covmat_bool[pass_tcs].sum(axis=1)
e_f = covmat_bool[fail_tcs].sum(axis=1)
n_p = P - e_p
n_f = F - e_f
spectra = pd.DataFrame(
    {
        "line": covmat.index.to_list(),
        "e_p": e_p,
        "e_f": e_f,
        "n_p": n_p,
        "n_f": n_f,
    }
)

display(spectra)


line  \
file                                                  function                    line                                                                                              
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   (/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap, 174)   
                                                                                  177   (/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap, 177)   
                                                                                  199   (/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap, 199)   
                                                                                  203   (/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap, 203)   
                                                                                  205   (/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap, 205)   
...                                                                                                                                                                           ...   
/home/workspace/epan/dissectors/packet-isns.c         proto_reg_handoff_isns      1712  (/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns, 1712)               
                                                                                  1713  (/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns, 1713)               
                                                                                  1715  (/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns, 1715)               
                                                                                  1716  (/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns, 1716)               
                                                                                  1717  (/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns, 1717)               

                                                                                        e_p  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   33    
                                                                                  177   33    
                                                                                  199   33    
                                                                                  203   33    
                                                                                  205   33    
...                                                                                     ..    
/home/workspace/epan/dissectors/packet-isns.c         proto_reg_handoff_isns      1712  33    
                                                                                  1713  33    
                                                                                  1715  33    
                                                                                  1716  33    
                                                                                  1717  33    

                                                                                        e_f  \
file                                                  function                    line        
/home/workspace/epan/dissectors/packet-gluster_pmap.c proto_register_gluster_pmap 174   1     
                                                                                  177   1     
                                                                                  199   1     
                                                                                  203   1     
             

# Compute suspiciousness scores

In [4]:
def ochiai(e_f, e_p, n_f, n_p):
    return e_f / np.sqrt(e_p + e_f) / np.sqrt(e_f + n_f) if e_f else 0.0


def op2(e_f, e_p, n_f, n_p):
    return e_f - e_p / (e_p + n_p + 1)

spectra["Sop2"] = spectra.apply(
    lambda row: op2(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Sochiai"] = spectra.apply(
    lambda row: ochiai(row.e_f, row.e_p, row.n_f, row.n_p), axis=1
)
spectra["Rop2"] = spectra.Sop2.rank(ascending=False, method="max")
spectra["Rochiai"] = spectra.Sochiai.rank(ascending=False, method="max")


# Line level SBFL

In [5]:
for fault_meth in fault_meth2idx:
    fault_idxs_of_meth = fault_meth2idx[fault_meth]
    print("fault meth:", fault_meth)
    for fault_idx in fault_idxs_of_meth:
        print("fault idx:", tuple(fault_idx))
    print("SBFL score of faulty method:")
    display(spectra.loc[fault_idxs_of_meth].reset_index(drop=True))

print("Top 5 SBFL scores:")
print("Op2:")
display(
    spectra.sort_values("Sop2", ascending=False).head(5).reset_index(drop=True)
)
print("Ochiai:")
display(
    spectra.sort_values("Sochiai", ascending=False)
    .head(5)
    .reset_index(drop=True)
)


fault meth: ('/home/workspace/epan/wslua/wslua_byte_array.c', 'ByteArray_base64_decode')
fault idx: ('/home/workspace/epan/wslua/wslua_byte_array.c', 'ByteArray_base64_decode', 267)
fault idx: ('/home/workspace/epan/wslua/wslua_byte_array.c', 'ByteArray_base64_decode', 266)
fault idx: ('/home/workspace/epan/wslua/wslua_byte_array.c', 'ByteArray_base64_decode', 265)
fault idx: ('/home/workspace/epan/wslua/wslua_byte_array.c', 'ByteArray_base64_decode', 262)
fault idx: ('/home/workspace/epan/wslua/wslua_byte_array.c', 'ByteArray_base64_decode', 268)
fault idx: ('/home/workspace/epan/wslua/wslua_byte_array.c', 'ByteArray_base64_decode', 261)
SBFL score of faulty method:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode, 267)",0,1,33,0,1.0,1.0,4981.0,4981.0
1,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode, 266)",0,1,33,0,1.0,1.0,4981.0,4981.0
2,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode, 265)",0,1,33,0,1.0,1.0,4981.0,4981.0
3,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode, 262)",0,1,33,0,1.0,1.0,4981.0,4981.0
4,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode, 268)",0,1,33,0,1.0,1.0,4981.0,4981.0
5,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode, 261)",0,1,33,0,1.0,1.0,4981.0,4981.0


Top 5 SBFL scores:
Op2:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/wiretap/k12.c, k12_open, 823)",0,1,33,0,1.0,1.0,4981.0,4981.0
1,"(/home/workspace/epan/wslua/wslua_pinfo.c, Pinfo_get_lo, 382)",0,1,33,0,1.0,1.0,4981.0,4981.0
2,"(/home/workspace/epan/wslua/wslua_pinfo.c, Pinfo_get_private, 335)",0,1,33,0,1.0,1.0,4981.0,4981.0
3,"(/home/workspace/epan/wslua/wslua_pinfo.c, Pinfo_get_private, 334)",0,1,33,0,1.0,1.0,4981.0,4981.0
4,"(/home/workspace/epan/wslua/wslua_pinfo.c, Pinfo_get_columns, 330)",0,1,33,0,1.0,1.0,4981.0,4981.0


Ochiai:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai
0,"(/home/workspace/wiretap/k12.c, k12_open, 823)",0,1,33,0,1.0,1.0,4981.0,4981.0
1,"(/home/workspace/epan/wslua/wslua_pinfo.c, Pinfo_get_lo, 382)",0,1,33,0,1.0,1.0,4981.0,4981.0
2,"(/home/workspace/epan/wslua/wslua_pinfo.c, Pinfo_get_private, 335)",0,1,33,0,1.0,1.0,4981.0,4981.0
3,"(/home/workspace/epan/wslua/wslua_pinfo.c, Pinfo_get_private, 334)",0,1,33,0,1.0,1.0,4981.0,4981.0
4,"(/home/workspace/epan/wslua/wslua_pinfo.c, Pinfo_get_columns, 330)",0,1,33,0,1.0,1.0,4981.0,4981.0


# Method level SBFL

In [6]:
spectra["file"] = spectra.index.map(lambda x: x[0])
spectra["func"] = spectra.index.map(lambda x: x[1])
spectra["line"] = spectra.index.map(lambda x: x[2])
# (file, func) is meth
spectra["meth"] = spectra.apply(lambda row: (row.file, row.func), axis=1)
spectra.reset_index(drop=True, inplace=True)
display(spectra)

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth
0,174,33,1,0,0,0.029412,0.171499,98416.0,98416.0,/home/workspace/epan/dissectors/packet-gluster_pmap.c,proto_register_gluster_pmap,"(/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap)"
1,177,33,1,0,0,0.029412,0.171499,98416.0,98416.0,/home/workspace/epan/dissectors/packet-gluster_pmap.c,proto_register_gluster_pmap,"(/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap)"
2,199,33,1,0,0,0.029412,0.171499,98416.0,98416.0,/home/workspace/epan/dissectors/packet-gluster_pmap.c,proto_register_gluster_pmap,"(/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap)"
3,203,33,1,0,0,0.029412,0.171499,98416.0,98416.0,/home/workspace/epan/dissectors/packet-gluster_pmap.c,proto_register_gluster_pmap,"(/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap)"
4,205,33,1,0,0,0.029412,0.171499,98416.0,98416.0,/home/workspace/epan/dissectors/packet-gluster_pmap.c,proto_register_gluster_pmap,"(/home/workspace/epan/dissectors/packet-gluster_pmap.c, proto_register_gluster_pmap)"
...,...,...,...,...,...,...,...,...,...,...,...,...
98537,1712,33,1,0,0,0.029412,0.171499,98416.0,98416.0,/home/workspace/epan/dissectors/packet-isns.c,proto_reg_handoff_isns,"(/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns)"
98538,1713,33,1,0,0,0.029412,0.171499,98416.0,98416.0,/home/workspace/epan/dissectors/packet-isns.c,proto_reg_handoff_isns,"(/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns)"
98539,1715,33,1,0,0,0.029412,0.171499,98416.0,98416.0,/home/workspace/epan/dissectors/packet-isns.c,proto_reg_handoff_isns,"(/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns)"
98540,1716,33,1,0,0,0.029412,0.171499,98416.0,98416.0,/home/workspace/epan/dissectors/packet-isns.c,proto_reg_handoff_isns,"(/home/workspace/epan/dissectors/packet-isns.c, proto_reg_handoff_isns)"


## Max aggregation

In [7]:
print("[Op2]")
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_op2 = spectra_max_op2[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sop2"]
]
spectra_max_op2["Rop2"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_op2.loc[spectra_max_op2.meth.isin(fault_meths)].reset_index(
        drop=True
    )
)
print("Top 20 methods:")
display(spectra_max_op2.head(20).reset_index(drop=True))
print()
print("[Ochiai]")
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai.drop_duplicates(subset="meth", keep="first", inplace=True)
spectra_max_ochiai = spectra_max_ochiai[
    ["meth", "line", "e_p", "e_f", "n_p", "n_f", "Sochiai"]
]
spectra_max_ochiai["Rochiai"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="max"
)
print("Faulty method:")
display(
    spectra_max_ochiai.loc[
        spectra_max_ochiai.meth.isin(fault_meths)
    ].reset_index(drop=True)
)
print("Top 20 methods:")
display(spectra_max_ochiai.head(20).reset_index(drop=True))


[Op2]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode)",276,0,1,33,0,1.0,683.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sop2,Rop2
0,"(/home/workspace/wsutil/report_message.c, report_failure)",54,0,1,33,0,1.0,683.0
1,"(/home/workspace/wsutil/nstime.c, nstime_sum)",127,0,1,33,0,1.0,683.0
2,"(/home/workspace/wsutil/filesystem.c, get_dirname)",154,0,1,33,0,1.0,683.0
3,"(/home/workspace/wiretap/wtap.c, wtap_sequential_close)",1319,0,1,33,0,1.0,683.0
4,"(/home/workspace/wiretap/visual.c, visual_open)",176,0,1,33,0,1.0,683.0
5,"(/home/workspace/wiretap/tnef.c, tnef_open)",24,0,1,33,0,1.0,683.0
6,"(/home/workspace/wiretap/snoop.c, snoop_open)",249,0,1,33,0,1.0,683.0
7,"(/home/workspace/wiretap/rfc7468.c, rfc7468_open)",80,0,1,33,0,1.0,683.0
8,"(/home/workspace/wiretap/rfc7468.c, read_complete_text_line)",41,0,1,33,0,1.0,683.0
9,"(/home/workspace/wiretap/radcom.c, radcom_open)",103,0,1,33,0,1.0,683.0



[Ochiai]
Faulty method:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode)",276,0,1,33,0,1.0,683.0


Top 20 methods:


,meth,line,e_p,e_f,n_p,n_f,Sochiai,Rochiai
0,"(/home/workspace/wsutil/report_message.c, report_failure)",54,0,1,33,0,1.0,683.0
1,"(/home/workspace/wsutil/nstime.c, nstime_sum)",127,0,1,33,0,1.0,683.0
2,"(/home/workspace/wsutil/filesystem.c, get_dirname)",154,0,1,33,0,1.0,683.0
3,"(/home/workspace/wiretap/wtap.c, wtap_sequential_close)",1319,0,1,33,0,1.0,683.0
4,"(/home/workspace/wiretap/visual.c, visual_open)",176,0,1,33,0,1.0,683.0
5,"(/home/workspace/wiretap/tnef.c, tnef_open)",24,0,1,33,0,1.0,683.0
6,"(/home/workspace/wiretap/snoop.c, snoop_open)",249,0,1,33,0,1.0,683.0
7,"(/home/workspace/wiretap/rfc7468.c, rfc7468_open)",80,0,1,33,0,1.0,683.0
8,"(/home/workspace/wiretap/rfc7468.c, read_complete_text_line)",41,0,1,33,0,1.0,683.0
9,"(/home/workspace/wiretap/radcom.c, radcom_open)",103,0,1,33,0,1.0,683.0


## Line2Method voting (weight: sbfl score / dense rank)

In [8]:
print("[Op2]")
ballot_op2 = {}
spectra_max_op2 = spectra.copy().sort_values(
    ["Sop2", "meth", "line"], ascending=False
)
spectra_max_op2["Rop2D"] = spectra_max_op2.Sop2.rank(
    ascending=False, method="dense"
)
spectra_max_op2["vote"] = spectra_max_op2.Sop2 / spectra_max_op2.Rop2D
for row in spectra_max_op2.itertuples():
    if row.meth not in ballot_op2:
        ballot_op2[row.meth] = row.vote
    else:
        ballot_op2[row.meth] += row.vote
L2M_op2 = pd.DataFrame(
    {"meth": list(ballot_op2.keys()), "vote": list(ballot_op2.values())}
)
L2M_op2["rank"] = L2M_op2.vote.rank(ascending=False, method="max")
L2M_op2.sort_values("rank", inplace=True)
L2M_op2.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(L2M_op2.loc[L2M_op2.meth == fault_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == fault_meth].reset_index(
            drop=True
        )
    )
print("Top 5 methods:")
display(L2M_op2.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_op2.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_op2.loc[L2M_op2.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_op2.loc[spectra_max_op2.meth == top_meth].reset_index(
            drop=True
        )
    )
print()
print("[Ochiai]")
ballot_ochiai = {}
spectra_max_ochiai = spectra.copy().sort_values(
    ["Sochiai", "meth", "line"], ascending=False
)
spectra_max_ochiai["RochiaiD"] = spectra_max_ochiai.Sochiai.rank(
    ascending=False, method="dense"
)
spectra_max_ochiai["vote"] = (
    spectra_max_ochiai.Sochiai / spectra_max_ochiai.RochiaiD
)
for row in spectra_max_ochiai.itertuples():
    if row.meth not in ballot_ochiai:
        ballot_ochiai[row.meth] = row.vote
    else:
        ballot_ochiai[row.meth] += row.vote
L2M_ochiai = pd.DataFrame(
    {"meth": list(ballot_ochiai.keys()), "vote": list(ballot_ochiai.values())}
)
L2M_ochiai["rank"] = L2M_ochiai.vote.rank(ascending=False, method="max")
L2M_ochiai.sort_values("rank", inplace=True)
L2M_ochiai.reset_index(drop=True, inplace=True)
for fault_meth in fault_meths:
    print("Faulty method:")
    display(
        L2M_ochiai.loc[L2M_ochiai.meth == fault_meth].reset_index(drop=True)
    )
    print("Voters:")
    display(
        spectra_max_ochiai.loc[
            spectra_max_ochiai.meth == fault_meth
        ].reset_index(drop=True)
    )
print("Top 5 methods:")
display(L2M_ochiai.head(5).reset_index(drop=True))
for idx, top_meth in enumerate(L2M_ochiai.head(5).meth.tolist(), 1):
    print(f"Top {idx}:")
    display(L2M_ochiai.loc[L2M_ochiai.meth == top_meth].reset_index(drop=True))
    print("Voters:")
    display(
        spectra_max_ochiai.loc[spectra_max_ochiai.meth == top_meth].reset_index(
            drop=True
        )
    )


[Op2]
Faulty method:


,meth,vote,rank
0,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode)",15.0,128.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,Rop2D,vote
0,276,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_byte_array.c,ByteArray_base64_decode,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode)",1.0,1.0
1,275,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_byte_array.c,ByteArray_base64_decode,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode)",1.0,1.0
2,272,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_byte_array.c,ByteArray_base64_decode,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode)",1.0,1.0
3,271,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_byte_array.c,ByteArray_base64_decode,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode)",1.0,1.0
4,270,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_byte_array.c,ByteArray_base64_decode,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode)",1.0,1.0
5,268,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_byte_array.c,ByteArray_base64_decode,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode)",1.0,1.0
6,267,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_byte_array.c,ByteArray_base64_decode,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode)",1.0,1.0
7,266,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_byte_array.c,ByteArray_base64_decode,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode)",1.0,1.0
8,265,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_byte_array.c,ByteArray_base64_decode,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode)",1.0,1.0
9,264,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_byte_array.c,ByteArray_base64_decode,"(/home/workspace/epan/wslua/wslua_byte_array.c, ByteArray_base64_decode)",1.0,1.0


Top 5 methods:


...

,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,571,1,1,32,0,0.970588,0.707107,7651.0,7651.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553
1,570,1,1,32,0,0.970588,0.707107,7651.0,7651.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553
2,569,1,1,32,0,0.970588,0.707107,7651.0,7651.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553
3,568,1,1,32,0,0.970588,0.707107,7651.0,7651.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553
4,567,1,1,32,0,0.970588,0.707107,7651.0,7651.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
176,335,1,1,32,0,0.970588,0.707107,7651.0,7651.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553
177,330,1,1,32,0,0.970588,0.707107,7651.0,7651.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553
178,328,1,1,32,0,0.970588,0.707107,7651.0,7651.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553
179,327,1,1,32,0,0.970588,0.707107,7651.0,7651.0,/home/workspace/epan/reassemble_test.c,test_fragment_add_seq_partial_reassembly,"(/home/workspace/epan/reassemble_test.c, test_fragment_add_seq_partial_reassembly)",2.0,0.353553


Top 5:


,meth,vote,rank
0,"(/home/workspace/epan/wslua/wslua_struct.c, Struct_unpack)",62.0,5.0


Voters:


,line,e_p,e_f,n_p,n_f,Sop2,Sochiai,Rop2,Rochiai,file,func,meth,RochiaiD,vote
0,532,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_struct.c,Struct_unpack,"(/home/workspace/epan/wslua/wslua_struct.c, Struct_unpack)",1.0,1.0
1,531,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_struct.c,Struct_unpack,"(/home/workspace/epan/wslua/wslua_struct.c, Struct_unpack)",1.0,1.0
2,529,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_struct.c,Struct_unpack,"(/home/workspace/epan/wslua/wslua_struct.c, Struct_unpack)",1.0,1.0
3,527,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_struct.c,Struct_unpack,"(/home/workspace/epan/wslua/wslua_struct.c, Struct_unpack)",1.0,1.0
4,520,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_struct.c,Struct_unpack,"(/home/workspace/epan/wslua/wslua_struct.c, Struct_unpack)",1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,450,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_struct.c,Struct_unpack,"(/home/workspace/epan/wslua/wslua_struct.c, Struct_unpack)",1.0,1.0
58,449,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_struct.c,Struct_unpack,"(/home/workspace/epan/wslua/wslua_struct.c, Struct_unpack)",1.0,1.0
59,448,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_struct.c,Struct_unpack,"(/home/workspace/epan/wslua/wslua_struct.c, Struct_unpack)",1.0,1.0
60,447,0,1,33,0,1.0,1.0,4981.0,4981.0,/home/workspace/epan/wslua/wslua_struct.c,Struct_unpack,"(/home/workspace/epan/wslua/wslua_struct.c, Struct_unpack)",1.0,1.0


# 감상

Max aggregation으로 top이지만 그게 683개. line 수가 많은 편이 아니어서 voting으로도 성적이 좋지 않음